**Steps to Creating RDS Database**
1. Create an RDS MySQL database. Save the username (admin by default), password, and database ID that you create.
2. If you do not create a database ID, then you will have to create it in code, which can be seen in the second block.
3. You need to make sure the security permissions within the database on AWS has MySQL/Aurora access with port 0.0.0.0 to allow any inbound traffic given they have the cridentials

In [5]:
import mysql.connector
from datetime import datetime
from sqlalchemy import create_engine, text, Column, Integer, String,ForeignKey, UniqueConstraint, Table, MetaData, DateTime, Float

from sqlalchemy.orm import declarative_base, sessionmaker
# from sqlalchemy.sql import text

# Create the database if it doesn't exist
from sqlalchemy.orm import sessionmaker, relationship
import warnings

host     = 'finance-dash.csufivv7vfmp.us-east-1.rds.amazonaws.com'
username = 'admin'
password = 'finance-dash'
database = 'finance-dash'

In [6]:
# Database class
class Database:

	def __init__(self):

		self.host     = 'finance-dash.csufivv7vfmp.us-east-1.rds.amazonaws.com'
		self.username = 'admin'
		self.password = 'finance-dash'
		self.database = 'finance-dash'

	def _create_connection(self, host, username, password):
		
		conn = mysql.connector.connect(
            host=host,
            user=username,
            password=password
        	)
	
		cursor = conn.cursor()

		return conn, cursor

	
	def _create_engine(self):
		return create_engine(f'mysql+mysqlconnector://{self.username}:{self.password}@{self.host}/{self.database}')
	
	def _create_session(self, engine):
		return sessionmaker(bind=engine)()


	def create_database(self, host, username, password, database):

		conn, cursor = self._create_connection()

		try:
			cursor.execute(f"CREATE DATABASE IF NOT EXISTS `{database}`")
			cursor.close()
			conn.close()
		except mysql.connector.Error as err:
			print(f"Error: {err}")

		return 

# engine = create_database(host, username, password, database)


	def create_stock_data(self, data):
		# Create a configured "Session" class
		engine  = self._create_engine()
		session = self._create_session(engine)
		# Session = 

		# Create a session
		# session = Session()

		for company_name in data.keys():
			company = Stocks(company_name = company_name)
			session.add(company)  # Add the new_skill object to the session
			session.commit()

		return 

	def insert_stock_prices(self, data):
		'''
		Parameters:
		- data

		Returns:
		- 

		'''
		engine  = self._create_engine()
		# Session = sessionmaker(bind=engine)
		session = self._create_session(engine)
		# # Create a session
		# session = Session()


		if isinstance(data, dict):
			for key in data.keys():
				company_data = data[key]
				time_stamp   = company_data['Date']
				open_price   = company_data['Open']
				high_price   = company_data['High']
				low_price    = company_data['Low']
				close_price  = company_data['Close']
				volume       = company_data['Volume']

				stock_entry  = StockData(company_name = key, 
						time_stam     = time_stamp,
						open_price    = open_price,
						high_price    = high_price,
						low_price     = low_price,
						close_price   = close_price,
						volume        = volume)
				
				session.add(stock_entry)
				session.commit()



		return 


	def show_tables(self, show_rows = False):
		engine  = self._create_engine()

		with engine.connect() as connection:
			result = connection.execute(text("SHOW TABLES"))
			for row in result:
				table_name = row[0]
				if show_rows:
					rows = connection.execute(text(f'SELECT * FROM {table_name}'))

					for r in rows:
						print(r)

**Show Tables**

In [7]:
db = Database()

TypeError: create_stock_data() missing 1 required positional argument: 'data'

In [9]:
# Create a declarative base class
db     = Database()
engine = db._create_engine()
Base   = declarative_base()

# Define a table using a class
class Stocks(Base):
    __tablename__ = 'companies'  # This sets the table name to 'skills'
    id            = Column(Integer, primary_key=True, autoincrement=True)
    company_name  = Column(String(50), nullable=False)

    # Establish the relationship on the Company side
    # company_relationship = relationship('StockData', back_populates='company_name')


class StockData(Base):
    __tablename__ = 'lifetime_stocks'
    id            = Column(Integer, primary_key = True, autoincrement = True)
    company_id    = Column(Integer, ForeignKey('companies.id'))
    time_stamp    = Column(DateTime, nullable = False)
    open_price    = Column(Float, nullable = False)
    high_price    = Column(Float, nullable = False)
    low_price     = Column(Float, nullable = False)
    close_price   = Column(Float, nullable = False)
    volume        = Column(Float, nullable = False)

    # company_name  = relationship('Stocks', back_populates = 'company_relationship')


# By default this calls any class that inherits from Stocks
Base.metadata.create_all(engine)

